In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

## data preprocessing

In [4]:
df=pd.read_csv("Datasets/AS2(t-12).csv")
data=df["clave"]
df.drop(columns=["clave"],axis=1,inplace=True)
df['hour(t)']=list(np.arange(13,24))+list(np.arange(0,24))*1307
df=df.reindex(columns=['var1(t-12)','var2(t-12)','var3(t-12)','var4(t-12)','var5(t-12)','var6(t-12)','var7(t-12)','var8(t-12)','hour(t)','obj'])

print(df)
print(data)

       var1(t-12)  var2(t-12)  var3(t-12)  var4(t-12)  var5(t-12)  var6(t-12)  \
0             799      1442.0       658.0        76.0         0.0        36.0   
1             894       903.0       458.0        76.0         0.0        36.0   
2             760       397.0       190.0        76.0         0.0        36.0   
3             669       312.0       147.0        76.0         0.0        36.0   
4             665       299.0       124.0        76.0         0.0        36.0   
...           ...         ...         ...         ...         ...         ...   
31374         261       260.0       114.0       218.0         0.0       170.0   
31375         395       420.0       170.0       218.0         0.0       170.0   
31376         459       590.0       228.0       218.0         0.0       170.0   
31377         631       696.0       305.0       218.0         0.0       170.0   
31378         743       852.0       353.0       218.0         0.0       170.0   

       var7(t-12) var8(t-12

In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['var8(t-12)']=le.fit_transform(df['var8(t-12)'])
# le.transform(df['var8(t-12)'])
print(df)

       var1(t-12)  var2(t-12)  var3(t-12)  var4(t-12)  var5(t-12)  var6(t-12)  \
0             799      1442.0       658.0        76.0         0.0        36.0   
1             894       903.0       458.0        76.0         0.0        36.0   
2             760       397.0       190.0        76.0         0.0        36.0   
3             669       312.0       147.0        76.0         0.0        36.0   
4             665       299.0       124.0        76.0         0.0        36.0   
...           ...         ...         ...         ...         ...         ...   
31374         261       260.0       114.0       218.0         0.0       170.0   
31375         395       420.0       170.0       218.0         0.0       170.0   
31376         459       590.0       228.0       218.0         0.0       170.0   
31377         631       696.0       305.0       218.0         0.0       170.0   
31378         743       852.0       353.0       218.0         0.0       170.0   

       var7(t-12)  var8(t-1

In [26]:
def slicing_data(data,rows):
    x=list()
    y=list()
    for i in np.arange(rows-1,len(data)):
        y.append(data.iloc[i][-1])
        x.append([data.iloc[i-3][:-1],data.iloc[i-2][:-1],data.iloc[i-1][:-1],data.iloc[i][:-1]])
    
    return np.array(x),np.array(y)


In [27]:
x,y=slicing_data(df,4)
print(x)
print(y)

[[[7.990e+02 1.442e+03 6.580e+02 ... 1.160e+02 1.000e+00 1.300e+01]
  [8.940e+02 9.030e+02 4.580e+02 ... 1.160e+02 1.000e+00 1.400e+01]
  [7.600e+02 3.970e+02 1.900e+02 ... 1.160e+02 1.000e+00 1.500e+01]
  [6.690e+02 3.120e+02 1.470e+02 ... 1.160e+02 1.000e+00 1.600e+01]]

 [[8.940e+02 9.030e+02 4.580e+02 ... 1.160e+02 1.000e+00 1.400e+01]
  [7.600e+02 3.970e+02 1.900e+02 ... 1.160e+02 1.000e+00 1.500e+01]
  [6.690e+02 3.120e+02 1.470e+02 ... 1.160e+02 1.000e+00 1.600e+01]
  [6.650e+02 2.990e+02 1.240e+02 ... 1.160e+02 1.000e+00 1.700e+01]]

 [[7.600e+02 3.970e+02 1.900e+02 ... 1.160e+02 1.000e+00 1.500e+01]
  [6.690e+02 3.120e+02 1.470e+02 ... 1.160e+02 1.000e+00 1.600e+01]
  [6.650e+02 2.990e+02 1.240e+02 ... 1.160e+02 1.000e+00 1.700e+01]
  [5.320e+02 3.620e+02 1.290e+02 ... 1.160e+02 1.000e+00 1.800e+01]]

 ...

 [[2.470e+02 1.650e+02 6.600e+01 ... 2.650e+02 3.000e+00 1.800e+01]
  [2.610e+02 2.600e+02 1.140e+02 ... 2.650e+02 3.000e+00 1.900e+01]
  [3.950e+02 4.200e+02 1.700e+02 ...

In [33]:
print(x.shape)
print(y.shape)

(31376, 4, 9)
(31376,)
